# Open A Restaurant in Seattle

### Introduction
To open a restaurant in Seattle, it's crucial to understand where to locate. Through this study, we could identify the best location to set our business. However, how could we identify the most suitable location? It's through the location analysis.\
\
Competition study is an important benchmark. We would need to learn how to stand up among so many competitors, then set up a marketing plan for our restaurant. Via this study, we would find the density of the restaurant around Pike Fish Market and different categories of restaurants nearby.

### Data
We would start on using Foursquare location data to find out all restaurant in Seattle. From those locations, we could identify the main activities area. To identify the main activities area, we would combine the population data from the government website to identify the high population area.\
\
To find the optimal location, we select from the high population area. Observing the density of competitors, we would decide to select a location in CBD or residential area. In this step, we would using K-mean clustering to identify the most density area of restaurants.


### Methodology
Pike Fish Market is a tourist spot. I expect that is the high traffic area. Hence, I used Foursquare to geocode the location of it as the origin to identify other competitors. \
\
Then, I used search word ‘restaurant’ to call 30 restaurants within 500 radius of Pike Fish Market using Foursquare API. After request the data, useful columns set into data frame for further analysis.\
\
To understand how categories available in the dataset, the 30 places would be grouped and counted in a summary. I found there’s 17 categories of restaurant. Then, a dummy column is created based on the 17 categories for map visualization.\
\
For better visualization, I had created a map which each color represent the type of restaurant, example: red represent seafood restaurant using the dummy column previously created.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [24]:
# The code was removed by Watson Studio for sharing.

In [3]:
address = '86 Pike Pl, Seattle, WA 98101, United States' # address of Pike fish market

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.60883880255411 -122.34063186363127


In [4]:
search_query = 'restaurant'
radius = 500
print(search_query + ' .... OK!')

restaurant .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f676460096e8e43564dd1de'},
 'response': {'venues': [{'id': '454c474af964a520bb3c1fe3',
    'name': "Lowell's Restaurant",
    'location': {'address': '1519 Pike Pl',
     'crossStreet': 'at 1st',
     'lat': 47.6089372,
     'lng': -122.341346,
     'labeledLatLngs': [{'label': 'display',
       'lat': 47.6089372,
       'lng': -122.341346},
      {'label': 'entrance', 'lat': 47.608735, 'lng': -122.340856}],
     'distance': 54,
     'postalCode': '98101',
     'cc': 'US',
     'city': 'Seattle',
     'state': 'WA',
     'country': 'United States',
     'formattedAddress': ['1519 Pike Pl (at 1st)',
      'Seattle, WA 98101',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ce941735',
      'name': 'Seafood Restaurant',
      'pluralName': 'Seafood Restaurants',
      'shortName': 'Seafood',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
       'suffix': '.png'},
      'primary': True}],
    'deli

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",1909674,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/lowells-res...,False,454c474af964a520bb3c1fe3,1519 Pike Pl,...,"[1519 Pike Pl (at 1st), Seattle, WA 98101, Uni...","[{'label': 'display', 'lat': 47.6089372, 'lng'...",47.608937,-122.341346,NaN,98101,WA,Lowell's Restaurant,v-1600611424,147656490
1,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",1812853,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/maximilien-...,False,4846c47ff964a5206e501fe3,81A Pike St Ste A,...,"[81A Pike St Ste A, Seattle, WA 98101, United ...","[{'label': 'display', 'lat': 47.60820684827345...",47.608207,-122.340824,NaN,98101,WA,Maximilien Restaurant,v-1600611424,32986097
2,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",1573629,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/the-fisherm...,False,4a988ca3f964a520a92c20e3,1301 Alaskan Way,...,"[1301 Alaskan Way (On Pier 57), Seattle, WA 98...","[{'label': 'display', 'lat': 47.60612306667809...",47.606123,-122.342305,NaN,98101,WA,The Fisherman's Restaurant & Bar,v-1600611424,NaN
3,"[{'id': '52e81612bcbc57f1066b7a06', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,40b13b00f964a520a2f61ee3,1916 Post Aly,...,"[1916 Post Aly (at Stewart St), Seattle, WA 98...","[{'label': 'display', 'lat': 47.61038005434942...",47.610380,-122.342221,NaN,98101,WA,Kells Irish Restaurant & Pub,v-1600611424,NaN
4,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",1724897,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/long-provin...,False,49ed594df964a520e3671fe3,1901 2nd Ave,...,"[1901 2nd Ave (at Stewart St), Seattle, WA 981...","[{'label': 'display', 'lat': 47.61082072245036...",47.610821,-122.340689,NaN,98101,WA,Long Provincial Vietnamese Restaurant,v-1600611424,NaN


In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Lowell's Restaurant,Seafood Restaurant,1519 Pike Pl,US,Seattle,United States,at 1st,54,"[1519 Pike Pl (at 1st), Seattle, WA 98101, Uni...","[{'label': 'display', 'lat': 47.6089372, 'lng'...",47.608937,-122.341346,NaN,98101,WA,454c474af964a520bb3c1fe3
1,Maximilien Restaurant,French Restaurant,81A Pike St Ste A,US,Seattle,United States,NaN,71,"[81A Pike St Ste A, Seattle, WA 98101, United ...","[{'label': 'display', 'lat': 47.60820684827345...",47.608207,-122.340824,NaN,98101,WA,4846c47ff964a5206e501fe3
2,The Fisherman's Restaurant & Bar,Seafood Restaurant,1301 Alaskan Way,US,Seattle,United States,On Pier 57,327,"[1301 Alaskan Way (On Pier 57), Seattle, WA 98...","[{'label': 'display', 'lat': 47.60612306667809...",47.606123,-122.342305,NaN,98101,WA,4a988ca3f964a520a92c20e3
3,Kells Irish Restaurant & Pub,Irish Pub,1916 Post Aly,US,Seattle,United States,at Stewart St,208,"[1916 Post Aly (at Stewart St), Seattle, WA 98...","[{'label': 'display', 'lat': 47.61038005434942...",47.610380,-122.342221,NaN,98101,WA,40b13b00f964a520a2f61ee3
4,Long Provincial Vietnamese Restaurant,Vietnamese Restaurant,1901 2nd Ave,US,Seattle,United States,at Stewart St,220,"[1901 2nd Ave (at Stewart St), Seattle, WA 981...","[{'label': 'display', 'lat': 47.61082072245036...",47.610821,-122.340689,NaN,98101,WA,49ed594df964a520e3671fe3
5,Saigon Vietnamese Restaurant II,Vietnamese Restaurant,1529 6th Ave,US,Seattle,United States,at Pine St,530,"[1529 6th Ave (at Pine St), Seattle, WA 98101,...","[{'label': 'display', 'lat': 47.61203898469434...",47.612039,-122.335397,NaN,98101,WA,4a9c2ec7f964a520523620e3
6,Blarney Stone Pub & Restaurant,Pub,1416 1st Ave,US,Seattle,United States,Pike St,119,"[1416 1st Ave (Pike St), Seattle, WA 98101, Un...","[{'label': 'display', 'lat': 47.60822528947973...",47.608225,-122.339326,NaN,98101,WA,53275b21498e7dc5f6eb9c56
7,Diego's Mexican Restaurant,Mexican Restaurant,1420 1st Ave,US,Seattle,United States,NaN,106,"[1420 1st Ave, Seattle, WA 98101, United States]","[{'label': 'entrance', 'lat': 47.608386, 'lng'...",47.608440,-122.339348,NaN,98101,WA,4f329ee719836c91c7e98ad5
8,Saigon Restaurant,Vietnamese Restaurant,1916 Pike Pl,US,Seattle,United States,NaN,221,"[1916 Pike Pl, Seattle, WA 98101, United States]",NaN,47.610021,-122.343005,NaN,98101,WA,47f2414cf964a520964e1fe3
9,Retro Restaurant And Lounge,Lounge,216 Stewart St,US,Seattle,United States,Stewart Street and 3rd Ave,288,"[216 Stewart St (Stewart Street and 3rd Ave), ...","[{'label': 'display', 'lat': 47.61136890490712...",47.611369,-122.339823,NaN,98101,WA,51f86c01498e5f3d48939878


In [19]:
dataframe_filtered['dummies'] = dataframe_filtered['categories']
restaurant_type = {'Seafood Restaurant':0,'Hotel Bar':1,'Thai Restaurant':2,'Chinese Restaurant':3,'Italian Restaurant':4,'Pub':5,'Vietnamese Restaurant':6,'Breakfast Spot':7,'Food':8,'French Restaurant':9,'Sushi Restaurant':10,'Mexican Restaurant':11,'Irish Pub':12,'Lounge':13,'Cocktail Bar':14,'Financial or Legal Service':15,'Restaurant':16}
dataframe_filtered['dummies'] = [restaurant_type[item] for item in dataframe_filtered['dummies']] 
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,dummies
0,Lowell's Restaurant,Seafood Restaurant,1519 Pike Pl,US,Seattle,United States,at 1st,54,"[1519 Pike Pl (at 1st), Seattle, WA 98101, Uni...","[{'label': 'display', 'lat': 47.6089372, 'lng'...",47.608937,-122.341346,NaN,98101,WA,454c474af964a520bb3c1fe3,0
1,Maximilien Restaurant,French Restaurant,81A Pike St Ste A,US,Seattle,United States,NaN,71,"[81A Pike St Ste A, Seattle, WA 98101, United ...","[{'label': 'display', 'lat': 47.60820684827345...",47.608207,-122.340824,NaN,98101,WA,4846c47ff964a5206e501fe3,9
2,The Fisherman's Restaurant & Bar,Seafood Restaurant,1301 Alaskan Way,US,Seattle,United States,On Pier 57,327,"[1301 Alaskan Way (On Pier 57), Seattle, WA 98...","[{'label': 'display', 'lat': 47.60612306667809...",47.606123,-122.342305,NaN,98101,WA,4a988ca3f964a520a92c20e3,0
3,Kells Irish Restaurant & Pub,Irish Pub,1916 Post Aly,US,Seattle,United States,at Stewart St,208,"[1916 Post Aly (at Stewart St), Seattle, WA 98...","[{'label': 'display', 'lat': 47.61038005434942...",47.610380,-122.342221,NaN,98101,WA,40b13b00f964a520a2f61ee3,12
4,Long Provincial Vietnamese Restaurant,Vietnamese Restaurant,1901 2nd Ave,US,Seattle,United States,at Stewart St,220,"[1901 2nd Ave (at Stewart St), Seattle, WA 981...","[{'label': 'display', 'lat': 47.61082072245036...",47.610821,-122.340689,NaN,98101,WA,49ed594df964a520e3671fe3,6
5,Saigon Vietnamese Restaurant II,Vietnamese Restaurant,1529 6th Ave,US,Seattle,United States,at Pine St,530,"[1529 6th Ave (at Pine St), Seattle, WA 98101,...","[{'label': 'display', 'lat': 47.61203898469434...",47.612039,-122.335397,NaN,98101,WA,4a9c2ec7f964a520523620e3,6
6,Blarney Stone Pub & Restaurant,Pub,1416 1st Ave,US,Seattle,United States,Pike St,119,"[1416 1st Ave (Pike St), Seattle, WA 98101, Un...","[{'label': 'display', 'lat': 47.60822528947973...",47.608225,-122.339326,NaN,98101,WA,53275b21498e7dc5f6eb9c56,5
7,Diego's Mexican Restaurant,Mexican Restaurant,1420 1st Ave,US,Seattle,United States,NaN,106,"[1420 1st Ave, Seattle, WA 98101, United States]","[{'label': 'entrance', 'lat': 47.608386, 'lng'...",47.608440,-122.339348,NaN,98101,WA,4f329ee719836c91c7e98ad5,11
8,Saigon Restaurant,Vietnamese Restaurant,1916 Pike Pl,US,Seattle,United States,NaN,221,"[1916 Pike Pl, Seattle, WA 98101, United States]",NaN,47.610021,-122.343005,NaN,98101,WA,47f2414cf964a520964e1fe3,6
9,Retro Restaurant And Lounge,Lounge,216 Stewart St,US,Seattle,United States,Stewart Street and 3rd Ave,288,"[216 Stewart St (Stewart Street and 3rd Ave), ...","[{'label': 'display', 'lat': 47.61136890490712...",47.611369,-122.339823,NaN,98101,WA,51f86c01498e5f3d48939878,13


In [20]:
category = dataframe_filtered['categories'].nunique()
category

17

In [21]:
category_count = dataframe_filtered.groupby('categories')[['name']].count()
category_count['percentage'] = dataframe_filtered.groupby('categories')[['name']].count().apply(lambda x:100 * x / float(x.count()))
sorted_category_count = category_count.sort_values('percentage',ascending=False)
sorted_category_count

,name,percentage
categories,,
Seafood Restaurant,6,35.294118
Vietnamese Restaurant,3,17.647059
Thai Restaurant,3,17.647059
Restaurant,3,17.647059
Food,2,11.764706
French Restaurant,2,11.764706
Mexican Restaurant,1,5.882353
Sushi Restaurant,1,5.882353
Pub,1,5.882353


In [23]:
import matplotlib.cm as cm
import matplotlib.colors as colors

venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Pike Fish Market

x = np.arange(category)
ys = [i + x + (i*x)**2 for i in range(category)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add a red circle marker to represent the Pike Fish Market
folium.features.CircleMarker(
    [latitude, longitude],
    radius=3,
    color='red',
    popup='Pike Fish Market',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the category restaurants as different color circle markers
markers_colors = []
for lat, lng, poi, cat, dummies in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name, dataframe_filtered.categories, dataframe_filtered.dummies):
    label = folium.Popup(str(poi) + ' Category ' + str(cat), parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color=rainbow[dummies-1],
        popup=label,
        fill = True,
        fill_color=rainbow[dummies-1],
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Result
From the summary, around 35% (6 restaurants) are seafood restaurants and most of it are nearby the seaside. There’s a lot of Asian culture restaurant like Thai restaurant, Vietnamese restaurant, nearby the Pike Fish Market. However, most of them are distance from seaside. \
\
From this map, I observed that there’s high density of restaurant nearby Pike Fish Market. Most of them are seafood restaurant or western style restaurant, like French style restaurant. 

### Discussion
From the analysis, I would recommend to open the restaurant based on category. If we choose to open a seafood restaurant, seaside would be a good location, however, there’s a lot of rivals. \
\
If we choose an Asian style restaurant, further location study are required because most of them are opened far from the seaside and Pike Fish Market. There’s not much of Asian style restaurant open nearby Pike Fish Market. \
\
If the target customer prefer tried out Asian Food, open a restaurant close to Pike Fish Market, it won’t be a bad idea. However, we might need more effort to outstanding among the competitors around Pike Fish Market.

### Conclusion
Through this capstone project, I learnt how to use Foursquare API to obtain location information for analysis. Also, it’s help build up a better judgement to perform an analysis.\
\
From the analysis, I conclude that there’s a lot of competitors around Pike Fish Market. To be competitive, person who interested to open a restaurant around that area should learn the target customers and try hard to outstand among different categories restaurant. 